In [3]:
import pandas as pd

# Part A - Calculating loyalty points
On each day, there are 2 slots for each of which the loyalty points are to be calculated: S1 from 12am to 12pm S2 from 12pm to 12am" "Based on the above information and the data provided answer the following questions:

1. Find Playerwise Loyalty points earned by Players in the following slots:- a. 2nd October Slot S1 b. 16th October Slot S2 b. 18th October Slot S1 b. 26th October Slot S2
2. Calculate overall loyalty points earned and rank players on the basis of loyalty points in the month of October. In case of tie, number of games played should be taken as the next criteria for ranking.
3. What is the average deposit amount?
4. What is the average deposit amount per user in a month?
5. What is the average number of games played per user?"

In [4]:
# Load data from Excel sheets
df1 = pd.read_excel('modified Data.xlsx', sheet_name='User', parse_dates=['Datetime'])
df2 = pd.read_excel('modified Data.xlsx', sheet_name='Deposit', parse_dates=['Datetime'])
df3 = pd.read_excel('modified Data.xlsx', sheet_name='Withdrawal', parse_dates=['Datetime'])

In [5]:
df1.head()

,User ID,Games Played,Datetime
0,851,1,2022-01-10
1,717,1,2022-01-10
2,456,1,2022-01-10
3,424,1,2022-01-10
4,845,1,2022-01-10


In [6]:
df2.head()

,User Id,Datetime,Amount
0,357,2022-01-10 00:03:00,2000
1,776,2022-01-10 00:03:00,2500
2,492,2022-01-10 00:06:00,5000
3,803,2022-01-10 00:07:00,5000
4,875,2022-01-10 00:09:00,1500


In [7]:
df3.head()

,User Id,Datetime,Amount
0,190,2022-01-10 00:03:00,5872
1,159,2022-01-10 00:16:00,9540
2,164,2022-01-10 00:24:00,815
3,946,2022-01-10 00:29:00,23000
4,763,2022-01-10 00:40:00,9473


In [8]:
# Slot boundaries
s1_start = pd.Timestamp('12:00:00')
s1_end = pd.Timestamp('23:59:59')
s2_start = pd.Timestamp('00:00:00')
s2_end = pd.Timestamp('11:59:59')

In [14]:
def calculate_loyalty_points(action_type, amount=None, num_deposits=None, num_withdrawals=None, games_played=None):
    if action_type == 'deposit':
        return 0.01 * amount
    elif action_type == 'withdrawal':
        return 0.005 * amount
    elif action_type == 'deposit_withdrawal_diff':
        return 0.001 * max(num_deposits - num_withdrawals, 0)
    elif action_type == 'games_played':
        return 0.2 * games_played

In [17]:
loyalty_points = {}
for _, row in df1.iterrows():
    user_id = row['User ID']
    datetime = row['Datetime']
    games_played = row['Games Played']
    
    # Determine the slot
    slot = None
    if s1_start <= datetime <= s1_end:
        slot = 'S1'
    elif s2_start <= datetime <= s2_end:
        slot = 'S2'
    
    # Initialize loyalty points for the player if not already present
    if user_id not in loyalty_points:
        loyalty_points[user_id] = {'S1': 0, 'S2': 0}
    
    # Calculate loyalty points for games played
    if slot:
        loyalty_points[user_id][slot] += calculate_loyalty_points('games_played', games_played=games_played)

In [18]:
for _, row in df2.iterrows():
    user_id = row['User Id']
    amount = row['Amount']
    datetime = row['Datetime']
    
    # Determine the slot
    slot = None
    if s1_start <= datetime <= s1_end:
        slot = 'S1'
    elif s2_start <= datetime <= s2_end:
        slot = 'S2'
    
    # Calculate loyalty points for deposit
    if slot:
        loyalty_points[user_id][slot] += calculate_loyalty_points('deposit', amount=amount)

In [19]:
for _, row in df3.iterrows():
    user_id = row['User Id']
    amount = row['Amount']
    datetime = row['Datetime']
    
    # Determine the slot
    slot = None
    if s1_start <= datetime <= s1_end:
        slot = 'S1'
    elif s2_start <= datetime <= s2_end:
        slot = 'S2'
    
    # Calculate loyalty points for withdrawal
    if slot:
        loyalty_points[user_id][slot] += calculate_loyalty_points('withdrawal', amount=amount)

In [21]:
for user_id in loyalty_points.keys():
    num_deposits = df2[df2['User Id'] == user_id].shape[0]
    num_withdrawals = df3[df3['User Id'] == user_id].shape[0]
    
    for slot in loyalty_points[user_id].keys():
        loyalty_points[user_id][slot] += calculate_loyalty_points('deposit_withdrawal_diff', 
                                                                  num_deposits=num_deposits, 
                                                                  num_withdrawals=num_withdrawals)

In [23]:
loyalty_points_df = pd.DataFrame(loyalty_points).T

# Calculate overall loyalty points earned
loyalty_points_df['Overall Loyalty Points'] = loyalty_points_df['S1'] + loyalty_points_df['S2']

# Rank players based on overall loyalty points and number of games played
loyalty_points_df['Rank'] = loyalty_points_df['Overall Loyalty Points'].rank(ascending=False, method='min')
loyalty_points_df['Games Played'] = df1.groupby('User ID')['Games Played'].sum()

# Calculate average deposit amount
average_deposit_amount = df2['Amount'].mean()

# Calculate average deposit amount per user
average_deposit_per_user = df2.groupby('User Id')['Amount'].mean().mean()

# Calculate average number of games played per user
average_games_played_per_user = df1.groupby('User ID')['Games Played'].sum().mean()

In [25]:
print("Playerwise Loyalty points earned:")
print(loyalty_points_df[['S1', 'S2']])
print("\nOverall Loyalty points and player ranking:")
print(loyalty_points_df[['Overall Loyalty Points', 'Rank', 'Games Played']].sort_values(by='Rank'))
print("\nAverage deposit amount:", average_deposit_amount)
print("Average deposit amount per user in a month:", average_deposit_per_user)
print("Average number of games played per user:", average_games_played_per_user)

Playerwise Loyalty points earned:
        S1     S2
851  0.043  0.043
717  0.050  0.050
456  0.003  0.003
424  0.057  0.057
845  0.000  0.000
..     ...    ...
159  0.007  0.007
422  0.000  0.000
408  0.037  0.037
384  0.002  0.002
969  0.001  0.001

[1000 rows x 2 columns]

Overall Loyalty points and player ranking:
     Overall Loyalty Points   Rank  Games Played
97                    0.430    1.0           694
837                   0.280    2.0           335
618                   0.276    3.0          7084
193                   0.258    4.0             1
548                   0.258    4.0            31
..                      ...    ...           ...
265                   0.000  814.0            26
180                   0.000  814.0            12
388                   0.000  814.0             2
883                   0.000  814.0            26
634                   0.000  814.0            24

[1000 rows x 3 columns]

Average deposit amount: 5492.185399701801
Average deposit amount pe

# Part B - How much bonus should be allocated to leaderboard players?
After calculating the loyalty points for the whole month find out which 50 players are at the top of the leaderboard. The company has allocated a pool of Rs 50000 to be given away as bonus money to the loyal players.

Now the company needs to determine how much bonus money should be given to the players.

Should they base it on the amount of loyalty points? Should it be based on number of games? Or something else?

That’s for you to figure out.

Suggest a suitable way to divide the allocated money keeping in mind the following points:

1. Only top 50 ranked players are awarded bonus

In [26]:
# Calculate loyalty points for each player
loyalty_points = {}

# Loop through each player's gameplay data
for _, row in df1.iterrows():
    user_id = row['User ID']
    games_played = row['Games Played']
    # Calculate loyalty points for games played
    loyalty_points[user_id] = loyalty_points.get(user_id, 0) + (0.2 * games_played)

# Loop through deposit data to calculate loyalty points
for _, row in df2.iterrows():
    user_id = row['User Id']
    amount = row['Amount']
    # Calculate loyalty points for deposit
    loyalty_points[user_id] = loyalty_points.get(user_id, 0) + (0.01 * amount)

# Loop through withdrawal data to calculate loyalty points
for _, row in df3.iterrows():
    user_id = row['User Id']
    amount = row['Amount']
    # Calculate loyalty points for withdrawal
    loyalty_points[user_id] = loyalty_points.get(user_id, 0) + (0.005 * amount)

In [27]:
#top 50 players based on loyalty points
top_50_players = sorted(loyalty_points.items(), key=lambda x: x[1], reverse=True)[:50]

In [29]:
# Approach 1: Weighted Distribution based on Loyalty Points
total_loyalty_points = sum(loyalty_points.values())
bonus_distribution_1 = {user_id: (loyalty_points[user_id] / total_loyalty_points) * 50000 for user_id, _ in top_50_players}

In [30]:
# Approach 2: Combination of Loyalty Points and Number of Games Played
games_played_dict = df1.groupby('User ID')['Games Played'].sum().to_dict()
combined_scores = {user_id: loyalty_points[user_id] + games_played_dict.get(user_id, 0) for user_id, _ in top_50_players}
total_combined_score = sum(combined_scores.values())
bonus_distribution_2 = {user_id: (combined_scores[user_id] / total_combined_score) * 50000 for user_id, _ in top_50_players}

In [31]:
# Approach 3: Equal Distribution among Top 50 Players
equal_bonus_share = 50000 / 50
bonus_distribution_3 = {user_id: equal_bonus_share for user_id, _ in top_50_players}

In [32]:
# Approach 4: Hybrid Approach
# Define weights for loyalty points and games played
loyalty_points_weight = 0.7
games_played_weight = 0.3
hybrid_scores = {user_id: (loyalty_points_weight * loyalty_points[user_id] + 
                           games_played_weight * games_played_dict.get(user_id, 0)) 
                 for user_id, _ in top_50_players}
total_hybrid_score = sum(hybrid_scores.values())
bonus_distribution_4 = {user_id: (hybrid_scores[user_id] / total_hybrid_score) * 50000 for user_id, _ in top_50_players}

In [33]:
# Approach 1: # Print bonus distributions for each approach
print("Approach 1: Weighted Distribution based on Loyalty Points")
print(bonus_distribution_1)
print("\nApproach 2: Combination of Loyalty Points and Number of Games Played")
print(bonus_distribution_2)
print("\nApproach 3: Equal Distribution among Top 50 Players")
print(bonus_distribution_3)
print("\nApproach 4: Hybrid Approach")
print(bonus_distribution_4)

Approach 1: Weighted Distribution based on Loyalty Points
{634: 2893.785406371953, 99: 816.8029090032362, 672: 785.465567257278, 212: 766.1897309088905, 740: 663.0778638968852, 566: 661.0748322177528, 714: 578.6029753650718, 421: 533.121713879155, 369: 498.331226281604, 30: 485.03928960358064, 587: 470.7322902619355, 222: 460.7228267095069, 352: 450.0834789812505, 365: 443.6826477577914, 920: 432.6631261009452, 162: 430.86148478707065, 415: 424.677057606484, 569: 424.0207700665735, 786: 417.5251973602287, 2: 415.56737928495016, 238: 401.70388743149067, 992: 401.3231957898801, 28: 366.62365311305865, 538: 355.4476095527955, 208: 339.63285732232646, 989: 337.86642049389604, 978: 324.39725339160464, 915: 322.3600526530713, 678: 320.7197652308128, 78: 315.2837785079154, 909: 312.9730527231661, 182: 305.1364307208322, 93: 304.2347472088602, 200: 288.7382159154394, 259: 285.2736803773794, 306: 285.11491505086934, 344: 284.3832139808666, 601: 280.5328096709844, 515: 272.0616440756323, 294: 26

# Part C
Would you say the loyalty point formula is fair or unfair?
Can you suggest any way to make the loyalty point formula more robust?

Here are some considerations regarding loyalty point formula is fair or unfair:
1. Transparency in the loyalty formula is essential.
2. The loyalty formula should treat all players equally and not prefer certain sorts of actions over others without justification.

Here are some suggestions to make loyalty formula more robust:
1. regularly collect feedback from players about their experiences with the loyalty point system.
2. Analyse player data to detect patterns and trends in activity, such as daily game time, total deposits, and so on.